### Libraries

In [2]:
!pip install tensorflow

     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 189.3/189.3 MB 2.6 MB/s eta 0:00:0000:0100:02
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 57.5/57.5 kB 2.7 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 440.8/440.8 kB 2.2 MB/s eta 0:00:00a 0:00:01
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 400.3/400.3 kB 3.7 MB/s eta 0:00:00a 0:00:01
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 8.8/8.8 MB 3.3 MB/s eta 0:00:0000:0100:01
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 1.7/1.7 MB 3.7 MB/s eta 0:00:00a 0:00:01
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 126.5/126.5 kB 2.6 MB/s eta 0:00:00a 0:00:01
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 65.5/65.5 kB 2.0 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 5.6/5.6 MB 3.2 MB/s eta 0:00:0000:0100:01
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 20.6/20.6 MB 2.5 MB/s eta 0:00:0000:0100:01
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 181.8/181.8 kB 3.6 MB/s eta 0:00:00a 0:00:01
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━

In [20]:
import random
import json
import pickle
import numpy as np

import nltk
from nltk.stem import WordNetLemmatizer

from tensorflow.keras import Sequential
from tensorflow.keras.layers import Dense , Activation , Dropout
from tensorflow.keras.optimizers import SGD
import tensorflow as tf
from tensorflow.keras.callbacks import LearningRateScheduler
from tensorflow.keras.models import load_model

In [3]:
nltk.download('omw-1.4')

[nltk_data] Downloading package omw-1.4 to
[nltk_data]     /Users/sherywalid/nltk_data...
[nltk_data]   Package omw-1.4 is already up-to-date!


True

In [4]:
from nltk.stem import PorterStemmer

stemmer = PorterStemmer()

### Extracting and Load Data

In [5]:
# load the json file with the bot responses
chat_data = json.loads(open("chatbot_data.json").read())

### Transforming Data

In [6]:
lemmatizer = WordNetLemmatizer()

In [7]:
chat_data

{'intents': [{'tag': 'greeting',
   'patterns': ['Hi',
    'How are you?',
    'Is anyone there?',
    'Hello',
    'Good day',
    "What's up",
    'how are ya',
    'heyy',
    'whatsup',
    '??? ??? ??'],
   'responses': ['Hello!',
    'Good to see you again!',
    'Hi there, how can I help?'],
   'context_set': ''},
  {'tag': 'goodbye',
   'patterns': ['cya',
    'see you',
    'bye bye',
    'See you later',
    'Goodbye',
    'I am Leaving',
    'Bye',
    'Have a Good day',
    'talk to you later',
    'ttyl',
    'i got to go',
    'gtg'],
   'responses': ['Sad to see you go :(',
    'Talk to you later',
    'Goodbye!',
    'Come back soon'],
   'context_set': ''},
  {'tag': 'creator',
   'patterns': ['what is the name of your developers',
    'what is the name of your creators',
    'what is the name of the developers',
    'what is the name of the creators',
    'who created you',
    'your developers',
    'your creators',
    'who are your developers',
    'developers',
  

In [8]:
words = []
classes = []
documents = []

ignore_symbols = [';',',','.','?','!','|','(',')']

In [9]:
# Iterate over all the intents in the chat data
for intent in chat_data['intents']:
    
    # Iterate over all the patterns in the current intent
    for pattern in intent['patterns']:
        
        # Tokenize the current pattern into a list of words
        words_list = nltk.tokenize.word_tokenize(pattern)
        
        # Add the words in the pattern to the global list of words
        words.extend(words_list)
        
        # Add the current pattern and its corresponding intent tag to the list of documents
        documents.append((words_list, intent['tag']))
        
        # Add the current intent tag to the list of classes if it's not already there
        if intent['tag'] not in classes:
            classes.append(intent['tag'])

In [10]:
print(documents)

[(['Hi'], 'greeting'), (['How', 'are', 'you', '?'], 'greeting'), (['Is', 'anyone', 'there', '?'], 'greeting'), (['Hello'], 'greeting'), (['Good', 'day'], 'greeting'), (['What', "'s", 'up'], 'greeting'), (['how', 'are', 'ya'], 'greeting'), (['heyy'], 'greeting'), (['whatsup'], 'greeting'), (['?', '?', '?', '?', '?', '?', '?', '?'], 'greeting'), (['cya'], 'goodbye'), (['see', 'you'], 'goodbye'), (['bye', 'bye'], 'goodbye'), (['See', 'you', 'later'], 'goodbye'), (['Goodbye'], 'goodbye'), (['I', 'am', 'Leaving'], 'goodbye'), (['Bye'], 'goodbye'), (['Have', 'a', 'Good', 'day'], 'goodbye'), (['talk', 'to', 'you', 'later'], 'goodbye'), (['ttyl'], 'goodbye'), (['i', 'got', 'to', 'go'], 'goodbye'), (['gtg'], 'goodbye'), (['what', 'is', 'the', 'name', 'of', 'your', 'developers'], 'creator'), (['what', 'is', 'the', 'name', 'of', 'your', 'creators'], 'creator'), (['what', 'is', 'the', 'name', 'of', 'the', 'developers'], 'creator'), (['what', 'is', 'the', 'name', 'of', 'the', 'creators'], 'creator'

In [11]:
# Lemmatize each word in the words list using WordNetLemmatizer
# and remove any words that are in the ignore_symbols list
words = [lemmatizer.lemmatize(w) for w in words if w not in ignore_symbols]

# Sort the list of words in lexicographic order and remove duplicates
words = sorted(set(words))

In [12]:
words

["'s",
 'AC',
 'AI/ML',
 'AI/Ml',
 'About',
 'Ac',
 'Are',
 'Bye',
 'Cafetaria',
 'Can',
 'Chemical',
 'Civil',
 'College',
 'Computer',
 'Contact',
 'Do',
 'Does',
 'Engineering',
 'Events',
 'Food',
 'Give',
 'Good',
 'Goodbye',
 'HOD',
 'Have',
 'Hello',
 'Hi',
 'Holiday',
 'How',
 'I',
 'IT',
 'Information',
 'Is',
 'Leaving',
 'Library',
 'Mechanical',
 'See',
 'Semester',
 'Sports',
 'Syllabus',
 'Technology',
 'Tell',
 'Thank',
 'Thanks',
 'UNI',
 'What',
 'Whats',
 'When',
 'Where',
 'Which',
 'Who',
 'Will',
 'You',
 'a',
 'about',
 'active',
 'activity',
 'address',
 'admision',
 'admission',
 'against',
 'allotment',
 'am',
 'an',
 'and',
 'antiragging',
 'any',
 'anyone',
 'are',
 'as',
 'asshole',
 'at',
 'attend',
 'automobile',
 'available',
 'average',
 'be',
 'between',
 'big',
 'bitch',
 'book',
 'boy',
 'branch',
 'bring',
 'building',
 'by',
 'bye',
 'cafetaria',
 'call',
 'called',
 'campus',
 'can',
 'canteen',
 'capacity',
 'case',
 'casuals',
 'ce',
 'chatting',

In [14]:
# Save the words variable to a file called 'word.pkl'
# using the pickle.dump() function and binary mode ('wb')
pickle.dump(words, open('word.pkl', 'wb'))

# Save the classes variable to a file called 'classes.pkl'
# using the pickle.dump() function and binary mode ('wb')
pickle.dump(classes, open('classes.pkl', 'wb'))

In [15]:
# Initialize an empty list called 'training'
training = []

# Create a list of zeros with length equal to the number of classes
# This will be used as a default output vector for each training example
output_empty = [0] * len(classes)

In [16]:
# Iterate over each document in the list of documents
for document in documents:
    
    # Initialize an empty list called 'bag'
    bag = []
    
    # Get the word patterns and lemmatize them
    word_patterns = document[0]
    word_patterns = [lemmatizer.lemmatize(word.lower()) for word in word_patterns]
    
    # Iterate over all the words in the global list 'words'
    # and add 1 to the 'bag' list if the word is in 'word_patterns'
    for word in words:
        if word in word_patterns:
            bag.append(1)
        else:
            bag.append(0)
    
    # Create an output row by copying the 'output_empty' list
    # and setting the appropriate index to 1 for the current document's intent class
    output_row = list(output_empty)
    output_row[classes.index(document[1])] = 1
    
    # Append the bag-of-words representation and the output row to the 'training' list
    training.append([bag, output_row])

In [17]:
# Shuffle the 'training' list randomly to ensure that the training examples are presented in a random order
random.shuffle(training)

# Convert the 'training' list to a NumPy array for easier indexing
training = np.array(training)

# Separate the input and output vectors into separate lists
train_x = list(training[:, 0])
train_y = list(training[:, 1])

/var/folders/7j/bl20_c316hs94pdnkbkjqmkr0000gn/T/ipykernel_4426/1474064510.py:5: VisibleDeprecationWarning: Creating an ndarray from ragged nested sequences (which is a list-or-tuple of lists-or-tuples-or ndarrays with different lengths or shapes) is deprecated. If you meant to do this, you must specify 'dtype=object' when creating the ndarray.
  training = np.array(training)


### Neural Network Model

In [18]:
# Initialize a new sequential neural network model
model = Sequential()

# Add a fully connected dense layer with 128 nodes, using ReLU activation function
# The input shape is the length of the 'train_x' list (i.e., the number of words in the vocabulary)
model.add(Dense(128, input_shape=(len(train_x[0]),), activation='relu'))

# Add a dropout layer with a rate of 0.5
# Dropout is a regularization technique that randomly drops out some nodes during training
model.add(Dropout(0.5))

# Add another fully connected dense layer with 64 nodes, using ReLU activation function
model.add(Dense(64, activation='relu'))

# Add another dropout layer with a rate of 0.5
model.add(Dropout(0.5))

# Add a fully connected dense layer with a number of nodes equal to the number of classes in 'train_y'
# Use softmax activation function to output a probability distribution over the classes
model.add(Dense(len(train_y[0]), activation='softmax'))

In [19]:
def lr_schedule(epoch, lr):
    """
    Learning rate schedule function that decreases the learning rate over time.
    The function takes an epoch number and a learning rate as input, and returns a new learning rate.
    """
    decay_rate = 1e-6
    return lr / (1 + decay_rate * epoch)

learning_rate = 0.01
optimizer = tf.keras.optimizers.SGD(learning_rate=learning_rate, momentum=0.9, nesterov=True)

lr_callback = LearningRateScheduler(lr_schedule)

model.compile(loss='categorical_crossentropy', optimizer=optimizer, metrics=['accuracy'])
hist = model.fit(np.array(train_x), np.array(train_y), epochs=200, batch_size=5, verbose=1, callbacks=[lr_callback])
model.save('chatbotmodel.h5', hist)
print("done")

Epoch 1/200
81/81 [==============================] - 0s 587us/step - loss: 3.5929 - accuracy: 0.0741 - lr: 0.0100
Epoch 2/200
81/81 [==============================] - 0s 558us/step - loss: 3.4205 - accuracy: 0.1235 - lr: 0.0100
Epoch 3/200
81/81 [==============================] - 0s 547us/step - loss: 3.1767 - accuracy: 0.2000 - lr: 0.0100
Epoch 4/200
81/81 [==============================] - 0s 558us/step - loss: 2.9157 - accuracy: 0.2617 - lr: 0.0100
Epoch 5/200
81/81 [==============================] - 0s 558us/step - loss: 2.7006 - accuracy: 0.2840 - lr: 0.0100
Epoch 6/200
81/81 [==============================] - 0s 554us/step - loss: 2.4247 - accuracy: 0.3630 - lr: 0.0100
Epoch 7/200
81/81 [==============================] - 0s 566us/step - loss: 2.2731 - accuracy: 0.4173 - lr: 0.0100
Epoch 8/200
81/81 [==============================] - 0s 552us/step - loss: 2.1003 - accuracy: 0.4642 - lr: 0.0100
Epoch 9/200
81/81 [==============================] - 0s 558us/step - loss: 1.8394 - accu

81/81 [==============================] - 0s 553us/step - loss: 0.2299 - accuracy: 0.9432 - lr: 0.0100
Epoch 73/200
81/81 [==============================] - 0s 555us/step - loss: 0.2356 - accuracy: 0.9333 - lr: 0.0100
Epoch 74/200
81/81 [==============================] - 0s 535us/step - loss: 0.2060 - accuracy: 0.9358 - lr: 0.0100
Epoch 75/200
81/81 [==============================] - 0s 532us/step - loss: 0.2566 - accuracy: 0.9086 - lr: 0.0100
Epoch 76/200
81/81 [==============================] - 0s 526us/step - loss: 0.2311 - accuracy: 0.9309 - lr: 0.0100
Epoch 77/200
81/81 [==============================] - 0s 526us/step - loss: 0.2137 - accuracy: 0.9333 - lr: 0.0100
Epoch 78/200
81/81 [==============================] - 0s 523us/step - loss: 0.2258 - accuracy: 0.9259 - lr: 0.0100
Epoch 79/200
81/81 [==============================] - 0s 529us/step - loss: 0.2025 - accuracy: 0.9210 - lr: 0.0100
Epoch 80/200
81/81 [==============================] - 0s 629us/step - loss: 0.1985 - accuracy

Epoch 143/200
81/81 [==============================] - 0s 565us/step - loss: 0.1276 - accuracy: 0.9556 - lr: 0.0099
Epoch 144/200
81/81 [==============================] - 0s 558us/step - loss: 0.0963 - accuracy: 0.9630 - lr: 0.0099
Epoch 145/200
81/81 [==============================] - 0s 551us/step - loss: 0.2153 - accuracy: 0.9333 - lr: 0.0099
Epoch 146/200
81/81 [==============================] - 0s 550us/step - loss: 0.1218 - accuracy: 0.9531 - lr: 0.0099
Epoch 147/200
81/81 [==============================] - 0s 559us/step - loss: 0.1442 - accuracy: 0.9580 - lr: 0.0099
Epoch 148/200
81/81 [==============================] - 0s 554us/step - loss: 0.1819 - accuracy: 0.9506 - lr: 0.0099
Epoch 149/200
81/81 [==============================] - 0s 612us/step - loss: 0.1586 - accuracy: 0.9358 - lr: 0.0099
Epoch 150/200
81/81 [==============================] - 0s 558us/step - loss: 0.1559 - accuracy: 0.9432 - lr: 0.0099
Epoch 151/200
81/81 [==============================] - 0s 553us/step - l

/Users/sherywalid/anaconda3/lib/python3.10/site-packages/keras/src/engine/training.py:3000: UserWarning: You are saving your model as an HDF5 file via `model.save()`. This file format is considered legacy. We recommend using instead the native Keras format, e.g. `model.save('my_model.keras')`.
  saving_api.save_model(


### Build ChatBot App

In [21]:
# Load the chatbot data from a JSON file and store it in a Python dictionary
chat_data = json.loads(open("chatbot_data.json").read())

# Load the vocabulary of words used to train the neural network model from a pickle file
words = pickle.load(open('word.pkl','rb'))

# Load the list of intent classes used to train the neural network model from a pickle file
classes = pickle.load(open('classes.pkl','rb'))

# Load the trained neural network model from a saved HDF5 file
model = load_model('chatbotmodel.h5')

In [24]:
def clean_sentence(sentence):
    """
    Tokenizes and lemmatizes the words in a sentence.

    Args:
    - sentence: A string representing the sentence to be tokenized and lemmatized.

    Returns:
    - A list of strings representing the lemmatized words in the sentence.
    """
    # Tokenize the sentence into individual words using the NLTK library
    sentence_words = nltk.word_tokenize(sentence)
    
    # Lemmatize each word using the WordNetLemmatizer from the NLTK library
    sentence_words = [lemmatizer.lemmatize(word) for word in sentence_words]
    
    # Return the lemmatized words
    return sentence_words

In [23]:

def bag_of_words(sentence):
    """
    Converts a sentence into a bag of words representation using the vocabulary of words.

    Args:
    - sentence: A string representing the sentence to be converted into a bag of words representation.

    Returns:
    - A 2D numpy array representing the bag of words representation of the sentence. The array has shape (1, len(words)),
      where `len(words)` is the length of the vocabulary of words used to train the neural network model.
      The array is a binary vector where each element is 1 if the corresponding word in the vocabulary is present in the
      sentence, and 0 otherwise.
    """
    # Tokenize and lemmatize the sentence using the `clean_sentence` function
    sentence_words = clean_sentence(sentence)
    
    # Initialize a binary vector of length `len(words)` with all elements set to 0
    bag = [0]*len(words)
    
    # Convert the sentence into a bag of words representation using the vocabulary of words
    for w in sentence_words:
        for i , word in enumerate(words):
            if word == w:
                bag[i] = 1
    
    # Return the bag of words representation as a 2D numpy array with shape (1, len(words))
    return np.array([bag])


In [25]:
def predict_class(sentence):
    """
    Predicts the intent of a sentence and returns a list of likely intents and their probabilities.

    Args:
    - sentence: A string representing the sentence for which the intent is to be predicted.

    Returns:
    - A list of dictionaries representing the likely intents and their probabilities. Each dictionary has two keys:
      - 'intent': A string representing the name of the predicted intent.
      - 'probability': A string representing the probability of the predicted intent, rounded to 2 decimal places.
    """
    # Convert the sentence into a bag of words representation using the `bag_of_words` function
    bow = bag_of_words(sentence)
    
    # Use the trained neural network model to predict the intent of the sentence
    res = model.predict(np.array(bow))[0]
    
    # Set an error threshold to filter out low-probability intents
    error_threshold = 0.25
    
    # Filter out low-probability intents
    result = [[i,r] for i, r in enumerate(res) if r > error_threshold]
    
    # Sort the intents by probability in descending order
    result.sort(key = lambda x : x[1], reverse = True)
    
    # Create a list of likely intents and their probabilities
    return_list = []
    for r in result:
        # Round the probability to 2 decimal places and store it as a string
        probability = str(round(r[1], 2))
        
        # Create a dictionary with keys 'intent' and 'probability' and add it to the return list
        return_list.append({'intent': classes[r[0]], 'probability' : probability})
        
    # Return the list of likely intents and their probabilities
    return return_list

In [26]:
def _get_response(ints, intents_json):
    """
    Retrieves a response for a given intent from a JSON file.

    Args:
    - ints: A list of dictionaries representing the likely intents and their probabilities. Each dictionary has two keys:
      - 'intent': A string representing the name of the predicted intent.
      - 'probability': A string representing the probability of the predicted intent, rounded to 2 decimal places.
    - intents_json: A dictionary containing the intents and their corresponding responses.

    Returns:
    - A string representing the response for the predicted intent. If the predicted intent is not found in the
      `intents_json` dictionary, a default response is returned.
    """
    try:
        # Get the tag of the predicted intent from the list of likely intents
        tag = ints[0]['intent']
        
        # Get the list of intents and their corresponding responses from the `intents_json` dictionary
        list_of_intents = intents_json['intents']
        
        # Find the intent that matches the predicted tag and choose a random response from its list of responses
        for i in list_of_intents:
            if i['tag'] == tag:
                result = random.choice(i['responses'])
                break
        else:
            # Default response when the predicted intent is not found in the `intents_json` dictionary
            result = "I don't have a response for that."
    except (IndexError, KeyError):
        # Default response when an error occurs while retrieving the response from the `intents_json` dictionary
        result = "I don't understand!"
        
    # Return the response for the predicted intent
    return result

In [ ]:
while True:
    message = input("")
    ints = predict_class(message)
    res = _get_response(ints, chat_data)
    print(res)

Heyy
1/1 [==============================] - 0s 15ms/step
Good to see you again!
any libraries
1/1 [==============================] - 0s 16ms/step
There is one huge and spacious library.timings are 8am to 6pm and for more visit <a target="blank" href="ADD LIBRARY DETAIL LINK">here</a>
thank you
1/1 [==============================] - 0s 16ms/step
welcome, anything else i can assist you with?
no bye
1/1 [==============================] - 0s 16ms/step
Sad to see you go :(
